In [1]:
#|export
import streamlit as st
from streamlit_jupyter import StreamlitPatcher
StreamlitPatcher().jupyter()
from nbdev.export import nb_export

# Set page configuration
st.set_page_config(layout="wide")

In [2]:
#|export
# generic libraries
import pandas as pd
import numpy as np
import time
import uuid

# pdf processing functions
import fitz
import pdf_utils

# NLP models
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline

In [3]:
#|export
# pipeline for base climate detector (stage-1 classifier)
name = "climatebert/distilroberta-base-climate-detector"
model = AutoModelForSequenceClassification.from_pretrained(name)
tokenizer = AutoTokenizer.from_pretrained(name, max_len=512)
detector_pipe = pipeline("text-classification", model=model, tokenizer=tokenizer)

In [4]:
#|export
# pipeline for sentiment classifier
name = "climatebert/distilroberta-base-climate-sentiment"
model = AutoModelForSequenceClassification.from_pretrained(name)
tokenizer = AutoTokenizer.from_pretrained(name, max_len=512)
sentiment_pipe = pipeline("text-classification", model=model, tokenizer=tokenizer)

In [5]:
#|export
# pipeline for commitment classifier
name = "climatebert/distilroberta-base-climate-commitment"
model = AutoModelForSequenceClassification.from_pretrained(name)
tokenizer = AutoTokenizer.from_pretrained(name, max_len=512)
commitment_pipe = pipeline("text-classification", model=model, tokenizer=tokenizer)

In [6]:
#|export
# pipeline for specificity classifier
name = "climatebert/distilroberta-base-climate-specificity"
model = AutoModelForSequenceClassification.from_pretrained(name)
tokenizer = AutoTokenizer.from_pretrained(name, max_len=512)
specificity_pipe = pipeline("text-classification", model=model, tokenizer=tokenizer)

In [7]:
#|export
# pipeline for TCFD classifier
name = "climatebert/distilroberta-base-climate-tcfd"
model = AutoModelForSequenceClassification.from_pretrained(name)
tokenizer = AutoTokenizer.from_pretrained(name, max_len=512)
tcfd_pipe = pipeline("text-classification", model=model, tokenizer=tokenizer)

In [8]:
#|export
# function to encapsulate all preprocessing steps (condensed version w/o OCR support)
def pdf_to_sentences(path, max_char, min_words=5):
    text = pdf_utils.true_pdf2text(path=path, stream=True)
    text = text[:max_char] # spacy has a 1M char limit
    sents = pdf_utils.text2sents(text=text, min_words=min_words)
    return sents

In [9]:
#|export
# returns climate-related sentences from a stage-1 classifier
def classify_s1(sents):
    # run stage-1 classifier
    s1_class = detector_pipe(sents, padding=True, truncation=True)
        
    # subset sents whose s1_class label is 'yes'
    y_index = [i for i, item in enumerate(s1_class) if item['label'] == 'yes']
    sents_s1 = [sents[i] for i in y_index]
    msg = ' '.join([str(len(sents_s1)), 'sentences are climate-related.', str(len(sents)-len(sents_s1)), 'sentences discarded.'])
    app.write(msg)
    
    return sents_s1

In [10]:
#|export
# runs a stage-2 classifier on sentences based on task
def classify_s2(sents, task):
    if task == "sentiment":
        s2_class = sentiment_pipe(sents, padding=True, truncation=True)
    elif task == "commitment":
        s2_class = commitment_pipe(sents, padding=True, truncation=True)
    elif task == "specificity":
        s2_class = specificity_pipe(sents, padding=True, truncation=True)
    else:
        s2_class = tcfd_pipe(sents, padding=True, truncation=True)
        
    # create df with texts and labels
    s2_labels = [x['label'] for x in s2_class]
    df = pd.DataFrame({"text": sents, task: s2_labels})
    
    # generate summary counts and plot
    counts = df.groupby(task).count()
    app.subheader(task.title())
    app.bar_chart(data=counts)
    summ = pd.DataFrame(counts).reset_index()
    
    return df, summ

In [11]:
#|export
# run stage-1 and multiple stage-2 classifiers > collate outputs
def classify(path, max_char=1000000):
    # run preprocessing steps
    sents = pdf_to_sentences(path, max_char, min_words=10)
    
    # NLP processing time per sentence based on test runs
    est_time = round(0.15*len(sents)/60) 
    msg = ("⏳Running classification... Estimated time remaining:", str(est_time), "mins")
    placeholder = app.empty()
    placeholder.write(" ".join(msg))

    # run stage-1 classifier
    sents_s1 = classify_s1(sents)
    
    # run s2 classifiers on s1 output
    df_sentiment, summ_sentiment = classify_s2(sents_s1, "sentiment")
    df_commitment, summ_commitment = classify_s2(sents_s1, "commitment")
    df_specificity, summ_specificity = classify_s2(sents_s1, "specificity")
    df_tcfd, summ_tcfd = classify_s2(sents_s1, "TCFD")
    
    # merge s2 labels into single df
    df_list = [df_sentiment, df_commitment, df_specificity, df_tcfd]
    tmp = [df.set_index(['text']) for df in df_list]
    df = pd.concat(tmp, axis=1).reset_index()
    
    # merge summary counts
    summ_list = [summ_sentiment, summ_commitment, summ_specificity, summ_tcfd]
    summ = pd.concat(summ_list, axis=1).fillna("")
    
    placeholder.empty()
    
    return(df, summ)

In [12]:
#|export
# calculate sustainability score
def sustainability_score(summ):
    n = int(summ.sum(axis=0, numeric_only=True)) # total no. of texts

    pct_nn = float((summ.iloc[1,1]+summ.iloc[2,1])/n) # Non-neutral
    pct_o = float(summ.iloc[1,1]/(summ.iloc[1,1]+summ.iloc[2,1])) # Opportunity vs. Risk
    pct_c = float(summ.iloc[1,3]/n) # Commitment = Y
    pct_sp = float(summ.iloc[1,5]/n) # Specific vs. Non-specific
    pct_m = float(summ.iloc[1,7]/n) # TCFD Metrics
    pct_st = float(summ.iloc[3,7]/n) # TCFD Strategy

    score = round(100*np.average([pct_nn*pct_o, pct_c, pct_sp, np.average([pct_m, pct_st])]), 2)
    return score

In [13]:
#|export
# generate content for About page
def setup_about_page(file):
    # demo data download
    with open(file, "rb") as pdf_file:
        demo_data = pdf_file.read()
    
    txt_demo = '''👋Welcome to the Corporate Greenwashing Detector tool!
    If you do not have a corporate sustainability report available, you can try out the app with a sample report.'''
    ack.markdown(txt_demo)
    ack.download_button(label="Download demo document",
                        data=demo_data,
                        file_name="Meta-2023-Sustainability-Report.pdf",
                        mime='application/octet-stream')    
    
    # acknowledgements
    ack.subheader(":star2: About ClimateBERT :star2:", divider="blue")
    ack_file = r'D:\Work\Climate NLP\proposals\Backdrop Build v5\acknowledgements.txt'
    with open(ack_file) as f:
        ack_text = f.read()
    ack.markdown(ack_text)
    
    # contact info
    txt_fb = '''📧Tell us about your experience with the Corporate Greenwashing Detector tool! Please send your 
    feedback and suggestions to <a href='mailto:ambar@scetti.org'> ambar@scetti.org</a>'''
    ack.markdown(txt_fb, unsafe_allow_html=True)

In [14]:
#|export
# function to keep count of app runs
def update_run_count():
    run_count_file = r'D:\Work\Climate NLP\web apps\run_count.txt'

    # read current value of run_count from file
    with open(run_count_file, "r") as file:
        run_count = int(file.read())

    # increment the count and write it back to the file
    run_count += 1
    with open(run_count_file, "w") as file:
        file.write(str(run_count))

    # st.write(f"This Streamlit app has been run {run_count} times.")
    return run_count    

In [15]:
#|export
st.title("Corporate Greenwashing Detector")

# user input widgets (form inside sidebar)
with st.sidebar:
    with st.form("user_inputs"):
        uploaded_file = st.file_uploader(label="Upload a PDF document",
                                         type=[".pdf"])
        submitted = st.form_submit_button("Analyze document")

# create separate tabs for analysis and acknowledgements
app, ack = st.tabs(["📈Analysis", "📢About"])

# setup About page
x = setup_about_page(file = r'D:\Work\Climate NLP\web apps\Meta-2023-Sustainability-Report.pdf')

# description for Analysis tab
app.markdown('''
This app uses AI-based Natural Language Processing to score companies on the ambition, credibility, and feasibility of climate change mitigation 
and adaptation commitments as articulated in their corporate sustainability report. A **Sustainability Score** for a company is calculated based on 
the following parameters: \n\n
❤️ :blue[Sentiment]: How many climate-related statements are classified as characterizing Risk, Opportunity, and Neutral?  
💍 :blue[Commitment]: How many statements demonstrate commitment to climate mitigation or adaptation goals?  
🎯 :blue[Specificity]: How many statements are specific versus non-specific in nature?  
📝 :blue[TCFD]: How many statements are classified under the TCFD pillars of Governance, Metrics, Risk, and Strategy?  
''')

if uploaded_file and submitted:
    start_time = time.time()
    x = update_run_count()
    
    # run 2-stage classification on a PDF (set max_char=1000 for testing)
    path = uploaded_file.read()
    df, summ = classify(path=path)
    
    if df is not None:
        ss = sustainability_score(summ)
        msg = " ".join(['Corporate Sustainability Score based on', uploaded_file.name, '=', str(ss)])
        app.write(msg)
        
        # print completion status
        app.write("Analysis completed! 👍")
        time_taken = round((time.time() - start_time)/60, 1)
        msg = " ".join(['Time taken:', str(time_taken), 'mins'])
        app.write(msg)

        # CSV file for user download
        file_name = uploaded_file.name.replace('pdf', 'csv')
        csv = df.to_csv(index=False).encode('utf-8')
        app.download_button(label="Download CSV output", 
                            data=csv, 
                            file_name=file_name)

# Corporate Greenwashing Detector

2024-07-29 19:29:35.317 
  command:

    streamlit run C:\Users\ambar\anaconda3\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


In [17]:
# exports this notebook to a .py file with streamlit statements added
nb_export("Corporate_Greenwashing_Detector.ipynb", lib_path="./", name="Corporate_Greenwashing_Detector")